In [ ]:
#pip install --upgrade numpy

In [ ]:
#pip install --force-reinstall pandas

In [1]:
#importing dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import pickle

/home/codespace/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# function for regression
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    #calculate duratiion time in minutes and storatinfg it in the vairabale "Duration"

    df['Duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['Duration'] = df['Duration'].apply(lambda x: x.total_seconds() / 60)
    df1 = df[((df.Duration > 1) & (df.Duration <= 60))]
    
    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    # Create a DataFrame df2 containing selected columns from df1
    df2 = df1[categorical + numerical].copy()

    # Add a new column 'Duration' from df1 to df2
    df2['Duration'] = df1['Duration']
    
    # Drop the target column from the dataframe
    X = df2.drop(columns=['Duration'], axis=1)

    # Get the target variable
    y = df2['Duration']
    
    categorical_columns = ['PULocationID', 'DOLocationID']  # List of categorical columns

    # Initialize OneHotEncoder
    one_hot_encoder = OneHotEncoder(sparse=True)


    # Fit and transform the categorical columns
    df_encoded = one_hot_encoder.fit_transform(df2[categorical_columns])
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(df_encoded, df2['Duration'], test_size=0.2, random_state=42)

    # Initialize the linear regression model
    model = LinearRegression()

    # Fit the model to the training data
    model.fit(X_train, y_train)
        
    # Predict on the test data
    y_pred = model.predict(X_test)


    # Calculate mean squared error
    mse = mean_squared_error(y_test, y_pred, squared=False)
    print("Mean Squared Error:", mse)
    
    return ["Mean Squared Error (test): " + str(mse)]

In [3]:
train= read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
validate= read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

Mean Squared Error: 7.65070954759519
Mean Squared Error: 7.783827641570997


In [19]:
def column(filename):
    df = pd.read_parquet(filename)
    shape = df.shape
    return df.shape 

In [20]:
column('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

(3066766, 19)

In [21]:
def duration(filename):
    df = pd.read_parquet(filename)
    df['Duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['Duration'] = df['Duration'].apply(lambda x: x.total_seconds() / 60)
    df1 = df[((df.Duration > 1) & (df.Duration <= 60))]
    std = np.std(df.Duration)
    return std

In [22]:
duration('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

42.5943442974141

In [4]:
# function for lasso regression
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    # Calculate duration time in minutes and store it in the variable "Duration"
    df['Duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['Duration'] = df['Duration'].apply(lambda x: x.total_seconds() / 60)
    df1 = df[((df.Duration > 1) & (df.Duration <= 60))]
    
    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    # Create a DataFrame df2 containing selected columns from df1
    df2 = df1[categorical + numerical].copy()

    # Add a new column 'Duration' from df1 to df2
    df2['Duration'] = df1['Duration']
    
    # Drop the target column from the dataframe
    X = df2.drop(columns=['Duration'], axis=1)

    # Get the target variable 
    y = df2['Duration']
    
    categorical_columns = ['PULocationID', 'DOLocationID']  # List of categorical columns

    # Initialize OneHotEncoder
    one_hot_encoder = OneHotEncoder(sparse=True)

    # Fit and transform the categorical columns
    df_encoded = one_hot_encoder.fit_transform(df2[categorical_columns])
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(df_encoded, df2['Duration'], test_size=0.2, random_state=42)

    alphas = [0.01, 0.001, 0.00001, 0.05]
    results = []

    for alpha in alphas:
        # Fit the Lasso model to your training data
        lasso_model = Lasso(alpha=alpha)
        lasso_model.fit(X_train, y_train)

        # Predict the target variable for your test data
        y_pred = lasso_model.predict(X_test)

        # Calculate mean squared error
        mse = mean_squared_error(y_test, y_pred, squared=False)
        print(f"Mean Squared Error for alpha={alpha}: {mse}")
        results.append((alpha, mse))
    
    return results


In [ ]:
train= read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
validate= read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

Mean Squared Error for alpha=0.01: 8.017330516339682
Mean Squared Error for alpha=0.001: 7.718834776863121


/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:609: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 40961536.3582398, tolerance: 23756.37349298848
  model = cd_fast.sparse_enet_coordinate_descent(


Mean Squared Error for alpha=1e-05: 7.650620611548999
Mean Squared Error for alpha=0.05: 8.451714672670622
Mean Squared Error for alpha=0.01: 8.117408601162468
Mean Squared Error for alpha=0.001: 7.853034292340794


In [2]:
from sklearn.model_selection import GridSearchCV
# function for lasso regression with gridsearch

def read_dataframe(filename):
    df = pd.read_parquet(filename)
    # Calculate duration time in minutes and store it in the variable "Duration"
    df['Duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    df['Duration'] = df['Duration'].apply(lambda x: x.total_seconds() / 60)
    df1 = df[((df.Duration > 1) & (df.Duration <= 60))]
    
    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    # Create a DataFrame df2 containing selected columns from df1
    df2 = df1[categorical + numerical].copy()

    # Add a new column 'Duration' from df1 to df2
    df2['Duration'] = df1['Duration']
    
    # Drop the target column from the dataframe
    X = df2.drop(columns=['Duration'], axis=1)

    # Get the target variable 
    y = df2['Duration']
    
    categorical_columns = ['PULocationID', 'DOLocationID']  # List of categorical columns

    # Initialize OneHotEncoder
    one_hot_encoder = OneHotEncoder(sparse=True)

    # Fit and transform the categorical columns
    df_encoded = one_hot_encoder.fit_transform(df2[categorical_columns])
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(df_encoded, df2['Duration'], test_size=0.2, random_state=42)

    # Define the hyperparameters grid
    param_grid = {'alpha': [0.1, 0.01,  0.0001, 0.000001, 0.5, 5, 1.0, 10.0, 50.0, 100.0, 500.0, 1000.0]}

    # Initialize the Lasso model
    lasso_model = Lasso()

    # Initialize GridSearchCV
    grid_search = GridSearchCV(estimator=lasso_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=1)

    # Fit GridSearchCV to the training data
    grid_search.fit(X_train, y_train)
    
    # Get the best estimator
    best_model = grid_search.best_estimator_

    # Predict the target variable for your test data using the best model
    y_pred = best_model.predict(X_test)

    # Calculate mean squared error
    mse = mean_squared_error(y_test, y_pred, squared=False)
    print("Mean Squared Error:", mse)

    return grid_search.best_params_


In [ ]:
train= read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
validate= read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [ ]:
'''param_grid = {'alpha': [0.1, 0.01,  0.0001, 0.000001, 0.5, 5, 1.0, 10.0, 50.0, 100.0, 500.0, 1000.0]}'''

In [ ]:
import os
'''with open('models/lin_reg/bin', 'wb') as f_out:
    pickle.dump((one_hot_encoder, model), f_out)
    
    with open('models/lin_reg/bin', 'wb') as f_out:
        pickle.dump((one_hot_encoder, lasso_model), f_out)
        
        
        with open('models/lin_reg/bin', 'wb') as f_out:
        pickle.dump((one_hot_encoder, best_model), f_out)
    
    return grid_search.best_params_'''
    
    
# Ensure the directory exists
    model_dir = 'models/grid_search'
    os.makedirs(model_dir, exist_ok=True)

# Save the OneHotEncoder and the best model using pickle
with open(os.path.join(model_dir, 'bin'), 'wb') as f_out:
    pickle.dump((one_hot_encoder, best_model), f_out)
    
print("Models have been saved to model.pkl")

In [ ]:
import os   
# Ensure the directory exists
model_dir = 'models/lasso_reg'
os.makedirs(model_dir, exist_ok=True)
    
with open(os.path.join(model_dir, 'bin'),'wb') as f_out:
        pickle.dump((one_hot_encoder, lasso_model), f_out)

print("Models have been saved to model.pkl")

In [ ]:
import os
# Ensure the directory exists
model_dir = 'models/lin_reg'
os.makedirs(model_dir, exist_ok=True)
    
with open(os.path.join(model_dir, 'bin'),'wb') as f_out:
        pickle.dump((one_hot_encoder, model), f_out)
print("Model have been saved to model.pkl")

In [ ]:
19, [], 98%, 2, 7.64, 7.81, {}